In [1]:
import datetime
import json
from pathlib import Path

import imageio.v2 as imageio
import numpy as np
import pandas as pd
import rasterio
import tqdm
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageFont

from estuary.util import masked_contrast_stretch

In [2]:
# Optional: try a nicer font; fall back to default if not available
try:
    FONT = ImageFont.truetype("/System/Library/Fonts/Supplemental/Arial Bold.ttf", 20)
except Exception:
    FONT = ImageFont.load_default()


def draw_label(
    img: Image.Image, text: str, color: tuple[int, int, int], add_border=True
) -> Image.Image:
    """Draw a semi-transparent banner with outlined text, and optional colored border."""
    draw = ImageDraw.Draw(img, "RGBA")
    w, h = img.size

    # Banner box
    pad_x, pad_y = 10, 8
    text_w, text_h = draw.textbbox((0, 0), text, font=FONT)[2:]
    box_w = min(w - 2 * pad_x, text_w + 2 * pad_x)
    box_h = text_h + 2 * pad_y

    # Top-left anchor for banner
    x0, y0 = pad_x, pad_y
    x1, y1 = x0 + box_w, y0 + box_h

    # Semi-transparent dark banner
    draw.rounded_rectangle([x0, y0, x1, y1], radius=10, fill=(0, 0, 0, 110))

    # Outlined text (stroke) for readability
    draw.text(
        (x0 + pad_x, y0 + pad_y),
        text,
        font=FONT,
        fill=(255, 255, 255, 255),
        stroke_width=2,
        stroke_fill=(0, 0, 0, 220),
    )

    # Optional border matching class color
    if add_border:
        draw.rectangle([0, 0, w - 1, h - 1], outline=color + (255,), width=4)

    return img

In [3]:
preds = pd.read_csv("/Users/kyledorman/data/estuary/preds.csv")
preds["acquired"] = pd.to_datetime(preds["acquired"], errors="coerce")
preds.head(3)

,index,region,udm_path,source_tif,label_idx,pred,conf,acquired
0,480,big_sur_river,/Users/kyledorman/data/estuary/dove/results/20...,/Users/kyledorman/data/estuary/dove/results/20...,0,0,0.995433,2019-01-01 18:25:38
1,481,big_sur_river,/Users/kyledorman/data/estuary/dove/results/20...,/Users/kyledorman/data/estuary/dove/results/20...,0,0,0.998330,2019-01-03 17:54:52
2,487,big_sur_river,/Users/kyledorman/data/estuary/dove/results/20...,/Users/kyledorman/data/estuary/dove/results/20...,0,0,0.998993,2019-01-04 17:54:40


In [4]:
CROP_PATH = Path("/Users/kyledorman/data/estuary/label_studio/region_crops.json")
region_crops = json.loads(CROP_PATH.read_bytes())

In [28]:
region = "little_sur"
start = datetime.datetime(year=2024, month=7, day=1)
end = datetime.datetime(year=2025, month=1, day=1)
crop = region_crops[region]
start_w, start_h, end_w, end_h = crop
w = end_w - start_w
h = end_h - start_h

gif_df = preds[(preds.region == region) & (preds.acquired > start) & (preds.acquired < end)]

len(gif_df)

55

In [43]:
save_path = Path(f"/Users/kyledorman/data/estuary/display/gifs/{region}/{start.date()}.mp4")
save_path.parent.mkdir(exist_ok=True, parents=True)

frames = []
for _, row in tqdm.tqdm(gif_df.iterrows(), total=len(gif_df)):
    pth = row.source_tif
    pred_name = "open" if row.pred == 0 else "close"
    pred_color = (44, 160, 44) if row.pred == 0 else (214, 39, 40)  # green/red
    conf_str = f"{row.conf:.2f}" if "conf" in gif_df.columns else "—"
    date_str = getattr(row, "acquired", None)
    if date_str is not None:
        # Parse YYYYMMDD or ISO-like strings robustly
        try:
            # if already datetime-like, this is a no-op; else try %Y%m%d
            dt = pd.to_datetime(date_str, format="%Y%m%d", errors="ignore")
            dt = pd.to_datetime(dt)  # ensure Timestamp
            date_disp = dt.strftime("%Y-%m-%d")
        except Exception:
            date_disp = str(date_str)
    else:
        date_disp = ""

    with rasterio.open(pth) as src:
        data = src.read(out_dtype=np.float32)[:, start_h:end_h, start_w:end_w]
        nodata = src.read(1, masked=True).mask[start_h:end_h, start_w:end_w]
    data = np.log10(data + 1)
    imgd = masked_contrast_stretch(data, ~nodata, p_low=1, p_high=99)
    rgb = imgd[[2, 1, 0]].transpose((1, 2, 0))
    img = Image.fromarray(np.array(np.clip(rgb * 255, 0, 255), dtype=np.uint8)).resize((256, 256))

    # Compose label text — include region/pred/conf/date as you like
    label_text = f"{pred_name}"
    if date_disp:
        label_text = f"{date_disp} • " + label_text

    img = draw_label(img, label_text, pred_color, add_border=True)

    frames.append(img)

# Convert each PIL frame to a NumPy array (imageio needs ndarray or PIL)
frame_arrays = [np.array(im.convert("RGB")) for im in frames]

  0%|                                                                                                                                                                                          | 0/55 [00:00<?, ?it/s]/var/folders/pr/1dt1qk511zdgzhfy1xskpqm00000gn/T/ipykernel_41190/156968114.py:15: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  dt = pd.to_datetime(date_str, format="%Y%m%d", errors="ignore")
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 55/55 [00:01<00:00, 46.22it/s]


In [30]:
# Write MP4 (H.264)
video_path = save_path
fps = 2
imageio.mimsave(
    video_path,
    frame_arrays,
    fps=fps,
    codec="libx264",  # H.264 for compatibility
    quality=10,  # 0 (lowest) - 10 (highest) for libx264
    macro_block_size=None,  # keeps original frame size
)
print(f"Saved video → {video_path}")

Saved video → /Users/kyledorman/data/estuary/display/gifs/little_sur/2024-07-01.mp4


In [31]:
from IPython.display import Video

Video(str(video_path), embed=True, width=600)

In [32]:
high_res = []
for pth in Path("/Users/kyledorman/data/estuary/skysat/results/").glob(
    "*/*/files/*_pansharpened_clip.tif"
):
    yearmonthday = pth.stem.split("_")[0]
    dt = pd.to_datetime(yearmonthday, format="%Y%m%d")
    high_res.append([pth, pth.parent.parent.name, dt])
high_res_df = pd.DataFrame(high_res, columns=["path", "region", "acquired"])

In [55]:
save_base = Path("/Users/kyledorman/data/estuary/display/skysat")

for region in preds.region.unique():
    pdf = preds[preds.region == region]
    hdf = high_res_df[high_res_df.region == region]
    # Work on sorted copies (required by merge_asof)
    hdf_s = hdf.sort_values("acquired").reset_index(drop=True)
    pdf_s = pdf.sort_values("acquired").reset_index(drop=True)
    # Nearest match within one week
    pairs = pd.merge_asof(
        hdf_s,
        pdf_s,
        on="acquired",
        direction="nearest",
        tolerance=pd.Timedelta("3D"),
        suffixes=("_h", "_p"),
    )
    # Keep only rows that found a match (otherwise columns from pdf will be NaN)
    pairs = pairs.dropna(subset=["path", "source_tif"])

    save = save_base / region
    save.mkdir(exist_ok=True, parents=True)

    crop = region_crops[region]
    start_w, start_h, end_w, end_h = crop
    w = end_w - start_w
    h = end_h - start_h

    for state in [0, 1]:
        for _, row in pairs[pairs.pred == state].iterrows():
            with rasterio.open(row.path) as src:
                data = src.read(out_dtype=np.float32)
                nodata = src.read(1, masked=True).mask
            data = np.log10(data + 1)
            imgd = masked_contrast_stretch(data, ~nodata, p_low=1, p_high=99)
            rgb = imgd[[2, 1, 0]].transpose((1, 2, 0))
            sky_img = Image.fromarray(np.array(np.clip(rgb * 255, 0, 255), dtype=np.uint8)).resize(
                (512, 512)
            )

            pred_name = "open" if row.pred == 0 else "close"

            fig, axes = plt.subplots(1, 2, figsize=(12, 6))
            axes[0].imshow(sky_img)
            axes[0].set_title(f"{pred_name} - {row.acquired.date()} - {region}")
            axes[0].axis("off")

            with rasterio.open(row.source_tif) as src:
                data = src.read(out_dtype=np.float32)[:, start_h:end_h, start_w:end_w]
                nodata = src.read(1, masked=True).mask[start_h:end_h, start_w:end_w]
            data = np.log10(data + 1)
            imgd = masked_contrast_stretch(data, ~nodata, p_low=1, p_high=99)
            rgb = imgd[[2, 1, 0]].transpose((1, 2, 0))
            img = Image.fromarray(np.array(np.clip(rgb * 255, 0, 255), dtype=np.uint8)).resize(
                (256, 256)
            )

            axes[1].imshow(img)
            axes[1].axis("off")

            plt.tight_layout()
            # plt.show()
            plt.savefig(save / f"{pred_name}_{row.acquired.date()}.png")
            plt.close()